In [ ]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.signal import argrelextrema

In [ ]:
now = datetime.now()

In [ ]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [ ]:
def eda(data):
    output=[]
    for col in data.columns:
        duplicatedvalue = data[col].duplicated().sum()
        duplicatedrows = data.duplicated().sum()
        missingvalue = np.sum(pd.isna(data[col]))
        uniquevalue = data[col].nunique()
        datatype = str(data[col].dtype)
        
        output.append([col, duplicatedvalue, duplicatedrows, missingvalue, uniquevalue, datatype])
        
    output = pd.DataFrame(output) 
    output.columns = ['Features', 'Duplicated Values', 'Duplicated Rows', 'Missing Values', 'Unique Values', 'Data Type']
    display(output)

In [ ]:
def perc_on_bar(plot, feature):
    total = len(feature)
    for p in ax.patches:
        percentage = "{:.1f}%".format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2 - 0.05
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size=12)
    plt.show()

In [ ]:
def viz(data, indice):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title="Neighbor: " + indice + " " + pair + " " + timeframe + " " + now.strftime("%Y-%m-%d %H:%M:%S"),
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    
    fig.write_image(path + "/" + indice + "_chart.png")
    
    fig.show()

In [ ]:
def market_order(instrument, units, take_profit, stop_loss):
    login_request_body = {
        "order": {
            "type": "MARKET",
            "instrument": instrument,
            "units": units,
            "timeInForce": "IOC",
            "positionFill": "DEFAULT",
            "takeProfitOnFill": {
                "price": take_profit
            },
            "stopLossOnFill": {
                "price": stop_loss
            }
        }
    }
    response = requests.post(provider_api_url, data=json.dumps(login_request_body),
                             headers=request_headers,
                             verify=False)
    response
    response.status_code

### The Configs for Run:

In [ ]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

# <font color='red'>Currency Pair</font>

In [ ]:
Load_10K_Records=True


asia = ['EUR_NZD','EUR_AUD','GBP_NZD','GBP_AUD','AUD_USD','AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD']

currency_pairs = ['EUR_USD','EUR_GBP','EUR_NZD','EUR_AUD','EUR_CHF','EUR_CAD',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD','AUD_USD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD',
                  'USD_CAD','USD_CHF','CAD_CHF']

currency_pairs = ["EUR_USD"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [ ]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

### Logging

In [ ]:
now = datetime.now()
for pair in currency_pairs:
    Log_Folder = now.strftime("%d-%m-%Y_%I-%M_%p")
    path = os.path.join(Log_Folder+"_"+pair)
    os.mkdir(path)

# Get Candlesticks Data

### Pricing & Spread Cost

In [ ]:
for pair in currency_pairs:
    pricing_params = (
        ('instruments', pair),
    )
    response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/pricing'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [ ]:
time = response['time']
ask = response['prices'][0]['closeoutAsk']
bid = response['prices'][0]['closeoutBid']
print ('Date:', time, 'Ask:', ask, 'Bid:', bid)

### Open Positions

In [ ]:
response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/openPositions'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()
response['positions']

###  Candlestick Data

In [ ]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [ ]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [ ]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [ ]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [ ]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [ ]:
data.shape

In [ ]:
data.head(2)

In [ ]:
data.tail(10)

## Simple Moving Average (SMA)

In [ ]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [ ]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [ ]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

## Price Range

In [ ]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [ ]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [ ]:
data.tail()

## Relative Strength Index (RSI)

In [ ]:
delta = data['Close'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
data['RSI'] = 100 - (100/(1 + rs))

## Calculate Average True Range (ATR)

In [ ]:
high_low = data['High'] - data['Low']
high_cp = np.abs(data['High'] - data['Close'].shift())
low_cp = np.abs(data['Low'] - data['Close'].shift())
df = pd.concat([high_low, high_cp, low_cp], axis=1)
true_range = np.max(df, axis=1)
data['ATR_14'] = true_range.rolling(14).mean()

# <font color='red'>CANDLE INDEX NUMBER</font>

In [ ]:
candle_no = len(data) - 2
candle_no

# Stop Loss & TakeProfit

In [ ]:
ATR = data.iloc[candle_no]['ATR_14']
CLOSED_PRICE = data.iloc[candle_no]['Close']
BUY_SL = (CLOSED_PRICE - ATR).round(5)
SELL_SL = (CLOSED_PRICE + ATR).round(5)
BUY_TP = (CLOSED_PRICE + ATR).round(5)
SELL_TP = (CLOSED_PRICE - ATR).round(5)

# Modeling

In [ ]:
data = pd.read_csv(filename)

### Feature Selection and Reduction

In [ ]:
data.columns

In [ ]:
data = data[['col_1','col_2','col_3','F_SMA_5','F_SMA_10']]

### Feature Being Fit to the Model

In [ ]:
data.head()

### Scaling using Standard Scaler

In [ ]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
            #print("Recommendation for {0}:\n".format(eurusd_data.index[candle_id]))
        else:
            #print("{0}: {1}, with distance of {2}".format(i,
            #                                               dataset.index[indices.flatten()[i]],
            #                                               distances.flatten()[i]))
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                           dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],dataset.iloc[indices.flatten()[i]]['col_2'],dataset.iloc[indices.flatten()[i]]['col_3'],
#                           dataset.iloc[indices.flatten()[i]]['col_4'],dataset.iloc[indices.flatten()[i]]['col_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_10'],
#                           dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
#                           dataset.iloc[indices.flatten()[i]]['RSI'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1','col_2','col_3',
#                      'col_4','col_5',
                      'F_SMA_5',
                      'F_SMA_10',
#                      'F_SMA_20',
#                      'RSI'
                     ]
    display (output)
    
    return indices, distances

### Top 5 Similar Candlesticks

In [ ]:
indices, distances = find_k_similar_candles (candle_no,data)

In [ ]:
indices = indices[0:1][0]

In [ ]:
indices

### Currnet Market/Candlestick

In [ ]:
closed_candle = "currnet_market_data.csv"
data = pd.read_csv(filename)
data = data.iloc[candle_no-30:candle_no+1]
data.to_csv(path + "/" + closed_candle, header = True, index = False)
viz(data, "current_market")

### Recommendations

In [ ]:
print(datetime.now())

In [ ]:
for indice in indices[1:10]:
    recommendation_log = "{}_data.csv".format(indice)
    data = pd.read_csv(filename) 
    data = data.iloc[indice:indice+5]
    data.to_csv(path + "/" + recommendation_log, header = True, index = False)
    
    print ('Neighbor:', indice, '|', '10K Records:', Load_10K_Records)
    viz(data, indice.astype(str))
    
    data['local_max'] = data['Close'][
        (data['Close'].shift(1) < data['Close']) & 
        (data['Close'].shift(-1) < data['Close'])]
    data['local_min'] = data['Close'][
        (data['Close'].shift(1) > data['Close']) &
        (data['Close'].shift(-1) > data['Close'])]
    
    max_idx = argrelextrema(data['Close'].values, np.greater, order=5)[0]
    min_idx = argrelextrema(data['Close'].values, np.less, order=5)[0]

    plt.figure(figsize=(15, 6))
    plt.plot(data['Close'], zorder=0)
    plt.scatter(data.iloc[max_idx].index, data.iloc[max_idx]['Close'], label='Maxima', s=100, color="green", marker='^')
    plt.scatter(data.iloc[min_idx].index, data.iloc[min_idx]['Close'], label='Minima', s=100, color="red", marker='v')
    plt.legend()
    plt.show()
    
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X)
    plt.figure(figsize=(15, 6))
    plt.scatter(X, Y)
    plt.plot(X, y_pred, color='red')
    plt.show()
    
    print(r2_score(Y, y_pred).round(2)*100, '% Fit')
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        print('Action: BUY')
        print('STOP LOSS:', BUY_SL, 'TAKE PROFIT:', BUY_TP)
    else:
        print('Action: SELL')
        print('STOP LOSS:', SELL_SL, 'TAKE PROFIT:', SELL_TP)


### Save the Recommendations

In [ ]:
currentpath = os.path.join(sys.path[0])
pngfiles = []
pngfiles = glob.glob(currentpath+"/"+path+"/*.png")
pdf = FPDF()

for pngfile in pngfiles:
    pdf.add_page()
    pdf.image(pngfile, w=200, h=130)
    
pdf.output(path+"/recommendations.pdf", "F")